In [4]:
import pandas as pd
import numpy as np

file_name = 'cryptocurrency.csv'

# Muat dataset
try:
    # Memuat file yang sudah diunggah
    df = pd.read_csv(file_name)
    print(f"✅ Dataset '{file_name}' berhasil dimuat dengan {len(df)} baris!")
except Exception as e:
    print(f"❌ Terjadi kesalahan saat memuat dataset: {e}")

✅ Dataset 'cryptocurrency.csv' berhasil dimuat dengan 119496 baris!


In [5]:
print("\n--- A. Discovery (Telaah Data) ---")

# Menampilkan 5 baris pertama
print("\n[A.1] 5 Baris Pertama:")
print(df.head())

# Menampilkan informasi umum (tipe data dan non-null count)
print("\n[A.2] Info Dataset Sebelum Cleaning:")
df.info()

# Menghitung Missing Values (Nilai Hilang)
print("\n[A.3] Jumlah Missing Values per Kolom:")
print(df.isnull().sum())

# Mendeteksi Duplikasi
print("\n[A.4] Jumlah Baris Duplikat:")
print(df.duplicated().sum())


--- A. Discovery (Telaah Data) ---

[A.1] 5 Baris Pertama:
             timestamp          name      symbol  price_usd         vol_24h  \
0  2025-10-11 12:23:08     LEO Token         leo      $9.59   $2,257,867.00   
1  2025-10-11 12:23:08      MemeCore           m      $2.24  $38,809,975.00   
2  2025-10-11 12:23:08  Figure Heloc  figr_heloc      $1.03  $61,601,743.00   
3  2025-10-11 12:23:08  Wrapped eETH       weeth  $4,132.48  $58,006,062.00   
4  2025-10-11 12:23:08          USDS        usds      $1.00  $53,468,731.00   

  total_vol  chg_24h   chg_7d       market_cap  
0     0.00%   +0.38%   -0.51%   $8,842,764,713  
1     0.00%   +8.30%   +7.11%   $3,779,986,616  
2     0.01%   +0.49%   +2.63%  $13,406,482,619  
3     0.01%  -11.69%  -14.79%  $10,218,284,869  
4     0.01%   -0.04%   +0.03%   $7,973,534,128  

[A.2] Info Dataset Sebelum Cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119496 entries, 0 to 119495
Data columns (total 9 columns):
 #   Column      Non-Nu

In [7]:
print("\n--- B & C. Structuring & Cleaning ---")

df_cleaned = df.copy()

# C.1. Menghapus Duplikasi (jika ada)
rows_before_drop = len(df_cleaned)
df_cleaned.drop_duplicates(inplace=True)
print(f"[C.1] Dihapus {rows_before_drop - len(df_cleaned)} baris duplikat.")

# C.2. Membersihkan dan Mengubah Kolom Tanggal/Waktu
# Mengubah kolom 'timestamp' ke tipe datetime
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'], errors='coerce')
print("[C.2] Kolom 'timestamp' dikonversi ke tipe datetime.")

# C.3. Membersihkan dan Mengubah Kolom Moneter ke Numerik
cols_to_clean = ['price_usd', 'vol_24h', 'market_cap']

for col in cols_to_clean:
    # Hapus karakter '$' dan ','
    # Gunakan .astype(str) untuk menghindari error pada nilai non-string (misalnya np.nan)
    df_cleaned[col] = df_cleaned[col].astype(str).str.replace(r'[$,"]', '', regex=True)

    # Mengubah ke tipe numerik. 'errors=coerce' akan mengubah nilai yang gagal di-parse menjadi NaN.
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
    print(f"[C.3] Kolom '{col}' dibersihkan dari '$', ',' dan diubah ke tipe float.")


# C.4. Menangani Missing Values (NaN) yang muncul setelah konversi
# Cek berapa banyak NaN yang muncul di kolom yang baru diubah
nan_after_conversion = df_cleaned[cols_to_clean].isnull().sum()
print("\n[C.4] Jumlah NaN setelah Konversi Numerik:")
print(nan_after_conversion[nan_after_conversion > 0])

# Strategi Imputasi: Ganti NaN pada price/vol/market_cap dengan median (nilai tengah)
for col in cols_to_clean:
    median_val = df_cleaned[col].median()
    df_cleaned[col].fillna(median_val, inplace=True)
print("\n[C.4] Nilai NaN pada kolom moneter diisi dengan median masing-masing kolom.")


--- B & C. Structuring & Cleaning ---
[C.1] Dihapus 0 baris duplikat.
[C.2] Kolom 'timestamp' dikonversi ke tipe datetime.
[C.3] Kolom 'price_usd' dibersihkan dari '$', ',' dan diubah ke tipe float.
[C.3] Kolom 'vol_24h' dibersihkan dari '$', ',' dan diubah ke tipe float.
[C.3] Kolom 'market_cap' dibersihkan dari '$', ',' dan diubah ke tipe float.

[C.4] Jumlah NaN setelah Konversi Numerik:
price_usd        36
vol_24h       97846
market_cap    97846
dtype: int64

[C.4] Nilai NaN pada kolom moneter diisi dengan median masing-masing kolom.


/tmp/ipython-input-3385644063.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(median_val, inplace=True)


In [8]:
print("\n--- D. Enriching (Pengayaan Fitur) ---")

# D.1. Ekstraksi Tanggal dan Waktu
df_cleaned['date'] = df_cleaned['timestamp'].dt.date
df_cleaned['hour'] = df_cleaned['timestamp'].dt.hour
print("[D.1] Kolom 'date' dan 'hour' berhasil ditambahkan dari 'timestamp'.")

# D.2. Membuat kolom 'Vol_per_Cap' (Rasio Volume 24 jam terhadap Market Cap)
df_cleaned['Vol_per_Cap'] = df_cleaned['vol_24h'] / df_cleaned['market_cap']
print("[D.2] Fitur 'Vol_per_Cap' (Volume/Market Cap) berhasil ditambahkan.")


--- D. Enriching (Pengayaan Fitur) ---
[D.1] Kolom 'date' dan 'hour' berhasil ditambahkan dari 'timestamp'.
[D.2] Fitur 'Vol_per_Cap' (Volume/Market Cap) berhasil ditambahkan.


In [9]:
print("\n--- E. Validating (Validasi Akhir) ---")

# E.1. Cek Ulang Missing Value
print("\n[E.1] Total Missing Values setelah Cleaning:")
print(df_cleaned.isnull().sum().sum()) # Seharusnya mendekati nol

# E.2. Cek Tipe Data Akhir
print("\n[E.2] Info Dataset Akhir:")
df_cleaned.info()

# E.3. Cek Statistik Deskriptif (memastikan format numerik sudah benar)
print("\n[E.3] Statistik Deskriptif Kolom Numerik Utama:")
print(df_cleaned[['price_usd', 'vol_24h', 'market_cap', 'Vol_per_Cap']].describe())


--- E. Validating (Validasi Akhir) ---

[E.1] Total Missing Values setelah Cleaning:
16

[E.2] Info Dataset Akhir:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119496 entries, 0 to 119495
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   timestamp    119496 non-null  datetime64[ns]
 1   name         119496 non-null  object        
 2   symbol       119496 non-null  object        
 3   price_usd    119496 non-null  float64       
 4   vol_24h      119496 non-null  float64       
 5   total_vol    119496 non-null  object        
 6   chg_24h      119496 non-null  object        
 7   chg_7d       119480 non-null  object        
 8   market_cap   119496 non-null  float64       
 9   date         119496 non-null  object        
 10  hour         119496 non-null  int32         
 11  Vol_per_Cap  119496 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int32(1), object(6)
memory usage: 

In [10]:
print("\n--- F. Publishing (Penyimpanan Data) ---")

file_output = 'cryptocurrency_bersih_output.csv'

# Menyimpan ke format CSV. index=False agar kolom index tidak ikut tersimpan.
df_cleaned.to_csv(file_output, index=False)

print(f"\n✅ Data bersih berhasil disimpan ke file: {file_output}")
print("Anda dapat mengunduh file ini dari panel File di sisi kiri Google Colab.")


--- F. Publishing (Penyimpanan Data) ---

✅ Data bersih berhasil disimpan ke file: cryptocurrency_bersih_output.csv
Anda dapat mengunduh file ini dari panel File di sisi kiri Google Colab.
